In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('ticks')

pd.set_option('display.max_columns', None)

%matplotlib inline

## Training data 

In [2]:
def preprocess_train(input_df):
    # make copy of df
    df = input_df.copy()
    
    # make columns lowercase and remove spaces
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '')

    # function to drop columns
    def dropcol(df, cols_to_drop):
        df.drop(cols_to_drop, axis = 1, inplace = True)
    
    # --------------------------------------- ADD NEW COLUMNS -------------------------------------------
    
    # 1 if feature present and 0 if feature absent
    pres = lambda x: 1 if x > 0 else 0
    
    # PORCH 
    col_porch = ['3ssnporch','enclosedporch','openporchsf','screenporch']
    df['porchpres'] = df[col_porch].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
    dropcol(df, col_porch)
    
    # POOL
    col_pool = ['poolqc','poolarea']
    df['poolpres'] = df['poolarea'].apply(pres)
    dropcol(df, col_pool)
    
    # GARAGE
    df['garagepres'] = df['garagearea'].apply(pres)
    
    # FIREPLACE
    df['fireplacepres'] = df['fireplaces'].apply(pres)
    
    # MASONRY VENEER
    df['masvnrpres'] = df['masvnrarea'].apply(pres)
    
    # BASEMENT
    df['bsmtpres'] = df['bsmtqual'].fillna('none').apply(lambda x: 0 if x == 'none' else 1)

    # LANDSLOPE
    df['landslope'] = df['landslope'].apply(lambda x: 'flat' if x == 'Gtl' else 'sloped')
    
    # MISCFEATURES
    df['shedpres'] = df['miscfeature'].apply(lambda x: 1 if x == 'Shed' else 0)
    dropcol(df, 'miscfeature')
    
    # EXTERIOR MATERIALS
    exterior_materials = {'MetalSd': 'Metal',
                          'HdBoard': 'Wood',
                          'Wd Sdng': 'Wood',
                          'Plywood': 'Wood',
                          'WdShing': 'Wood',
                          'CemntBd': 'Aggregate',
                          'Stucco': 'Aggregate',
                          'CBlock': 'Aggregate',
                          'AsphShn': 'Aggregate',
                          'ImStucc': 'Aggregate',
                          'BrkFace': 'Brick',
                          'BrkComm': 'Brick',
                          'AsbShng': 'Aesbestos',
                          'Stone': 'Stone',
                          'VinylSd': 'Vinyl'}
    df['exterior'] = df['exterior1st'].map(exterior_materials)
    dropcol(df, ['exterior1st','exterior2nd'])

    # AGE SOLD
    for index, val in enumerate(df['yearbuilt']):
        if val == df.loc[index, 'yrsold']:
            df.loc[index, 'age_sold'] = 0
        else:
            df.loc[index, 'age_sold'] = df.loc[index,'yrsold'] - val
    
    # REMODELLED
    # for index, val in enumerate(df['yearremod/add']):
    #     if val == df.loc[index,'yearbuilt']:
    #         df.loc[index,'remodelled'] = 0
    #     else:
    #         df.loc[index,'remodelled'] = 1
    # dropcol(df,['yearremod/add'])  
    
    # LIVING AREA
    df['2ndfloorpres'] = df['2ndflrsf'].apply(pres)     
    # get sum of 1stflr and 2ndflr area
    df['flrsf'] = df['1stflrsf']+df['2ndflrsf']
    # create bighouse column
    for index, val in enumerate(df['flrsf']):
        if val == df.loc[index, 'grlivarea']:
            df.loc[index, 'bighouse'] = 0
        else:
            df.loc[index, 'bighouse'] = 1
    dropcol(df, ['flrsf','1stflrsf','2ndflrsf'])
            
    # ---------------------------------------- IMPUTE NULL VALUES ---------------------------------------- 
    
    # LOT FRONTAGE
    # fill nan with neighbourhood median
    median_lotfrt_per_nbhood = df["lotfrontage"].groupby(df["neighborhood"])
    for nbhood, grouped in median_lotfrt_per_nbhood:
        index = (df['neighborhood'] == nbhood) & (df['lotfrontage'].isnull())
        df.loc[index,'lotfrontage'] = grouped.median()
    # for the few remaining rows from the neighborhoods without a median (i.e. if n < 3),
    # fill with overall lotfrontage median
    df['lotfrontage'] = df['lotfrontage'].fillna(df['lotfrontage'].median())

    # MASONRY VENEER
    # the NaN values in masvnrarea and masvnrtype are likely houses without masonry veneers
    df['masvnrarea'] = df['masvnrarea'].fillna(0.0)
    df['masvnrtype'] = df['masvnrtype'].fillna('None')
    
    # BASEMENT PROPERTIES
    # NaN values will be replaced with 'None' (no basement present)
    for col in ['bsmtqual','bsmtexposure','bsmtfintype1']:
        df[col] = df[col].fillna('None')
    # for the NaNs in the numerical columns, they will be replaced with 0 (since no basement)
    for col in ['bsmtfullbath','bsmtfinsf1','bsmtunfsf']:
        df[col] = df[col].fillna(0.0)

    # FIREPLACE
    # null value likely represent no fireplace
    df['fireplacequ'] = df['fireplacequ'].fillna('None')
    
    # GARAGE
    # null values likely represent no garage
    for col in ['garagetype','garagefinish','garagecond']:
        df[col] = df[col].fillna('NA')
    for col in ['garagearea','garagecars','garagearea']:
        df[col] = df[col].fillna(0.0)

    # ------------------------------ CONVERT ORDINAL CATEGORIES TO NUMERICAL ------------------------------
    
    # function to map new values
    def map_new_vals(colname,dictionary):
        df[colname] = df[colname].map(dictionary)
    
    # defining dictionaries for renaming
    lotshape_di = {'Reg': 0,
                   'IR1': 1,
                   'IR2': 2,
                   'IR3': 3}
    bsmtex_di = {'None': 0,
                'No': 1,
                'Mn': 2,
                'Av': 3,
                'Gd': 4}
    qual_di = {'Ex': 5,
               'Gd': 4,
               'TA': 3,
               'Fa': 2,
               'Po': 1,
               'None': 0}
    
    # map new values
    map_new_vals('lotshape', dictionary = lotshape_di)
    map_new_vals('bsmtexposure', dictionary = bsmtex_di)
    map_new_vals('exterqual', dictionary = qual_di)
    map_new_vals('bsmtqual', dictionary = qual_di)
    map_new_vals('heatingqc', dictionary = qual_di)
    map_new_vals('kitchenqual', dictionary = qual_di)
    map_new_vals('fireplacequ', dictionary = qual_di)

    # --------------------------------- CONVERT NUMERICAL TO CATEGORICAL --------------------------------
    
    df['mssubclass'] = df['mssubclass'].astype(str)

    # ------------------------------------------ DROP COLUMNS -------------------------------------------
        
    # SKEWED COLUMNS
    # columns with >80% single category or >80% zeros, plus pid columns
    col_to_drop = ['alley','miscval','lowqualfinsf','street','utilities','condition2','roofmatl',\
                   'heating','centralair','electrical','paveddrive','fence','saletype','bsmthalfbath',\
                   'bsmtfintype2','bsmtfinsf2','bsmtcond','extercond','garagequal']
    dropcol(df, col_to_drop)

    # COLLINEAR COLUMNS
    col_collinear = ['garagecars','totrmsabvgrd', 'totalbsmtsf','garageyrblt']
    dropcol(df, col_collinear) 

    # --------------------------------------- REMOVE OUTLIERS -----------------------------------------
    
    df.drop(df[df['grlivarea'] > 4_500].index, inplace = True)
    df.drop(df[df['lotfrontage'] > 300].index, inplace = True)
    df.drop(df[df['lotarea'] > 100_000].index, inplace = True)

    # ----------------------------------------- CLEANING UP -------------------------------------------
    
    # convert all NA to None
    df.replace('NA','None',inplace=True)

    # reset index
    df = df.reset_index()

    return df


## Test data 

In [3]:
def preprocess_test(input_df):
    # make copy of df
    df = input_df.copy()
    
    # make columns lowercase and remove spaces
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '')

    # function to drop columns
    def dropcol(df, cols_to_drop):
        df.drop(cols_to_drop, axis = 1, inplace = True)
    
    # --------------------------------------- ADD NEW COLUMNS -------------------------------------------
    
    # 1 if feature present and 0 if feature absent
    pres = lambda x: 1 if x > 0 else 0
    
    # PORCH 
    col_porch = ['3ssnporch','enclosedporch','openporchsf','screenporch']
    df['porchpres'] = df[col_porch].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
    dropcol(df, col_porch)
    
    # POOL
    col_pool = ['poolqc','poolarea']
    df['poolpres'] = df['poolarea'].apply(pres)
    dropcol(df, col_pool)
    
    # GARAGE
    df['garagepres'] = df['garagearea'].apply(pres)
    
    # FIREPLACE
    df['fireplacepres'] = df['fireplaces'].apply(pres)
    
    # MASONRY VENEER
    df['masvnrpres'] = df['masvnrarea'].apply(pres)
    
    # BASEMENT
    df['bsmtpres'] = df['bsmtqual'].fillna('none').apply(lambda x: 0 if x == 'none' else 1)

    # LANDSLOPE
    df['landslope'] = df['landslope'].apply(lambda x: 'flat' if x == 'Gtl' else 'sloped')
    
    # MISCFEATURES
    df['shedpres'] = df['miscfeature'].apply(lambda x: 1 if x == 'Shed' else 0)
    dropcol(df, 'miscfeature')
    
    # EXTERIOR MATERIALS
    exterior_materials = {'MetalSd': 'Metal',
                          'HdBoard': 'Wood',
                          'Wd Sdng': 'Wood',
                          'Plywood': 'Wood',
                          'WdShing': 'Wood',
                          'CemntBd': 'Aggregate',
                          'Stucco': 'Aggregate',
                          'CBlock': 'Aggregate',
                          'AsphShn': 'Aggregate',
                          'ImStucc': 'Aggregate',
                          'BrkFace': 'Brick',
                          'BrkComm': 'Brick',
                          'AsbShng': 'Aesbestos',
                          'Stone': 'Stone',
                          'VinylSd': 'Vinyl'}
    df['exterior'] = df['exterior1st'].map(exterior_materials)
    dropcol(df, ['exterior1st','exterior2nd'])

    # AGE SOLD
    for index, val in enumerate(df['yearbuilt']):
        if val == df.loc[index, 'yrsold']:
            df.loc[index, 'age_sold'] = 0
        else:
            df.loc[index, 'age_sold'] = df.loc[index,'yrsold'] - val
    
    # REMODELLED
    # for index, val in enumerate(df['yearremod/add']):
    #     if val == df.loc[index,'yearbuilt']:
    #         df.loc[index,'remodelled'] = 0
    #     else:
    #         df.loc[index,'remodelled'] = 1
    # dropcol(df,['yearremod/add'])  
    
    # LIVING AREA
    df['2ndfloorpres'] = df['2ndflrsf'].apply(pres)     
    # get sum of 1stflr and 2ndflr area
    df['flrsf'] = df['1stflrsf']+df['2ndflrsf']
    # create bighouse column
    for index, val in enumerate(df['flrsf']):
        if val == df.loc[index, 'grlivarea']:
            df.loc[index, 'bighouse'] = 0
        else:
            df.loc[index, 'bighouse'] = 1
    dropcol(df, ['flrsf','1stflrsf','2ndflrsf'])
            
    # ---------------------------------------- IMPUTE NULL VALUES ---------------------------------------- 
    
    # LOT FRONTAGE
    # fill nan with neighbourhood median
    median_lotfrt_per_nbhood = df["lotfrontage"].groupby(df["neighborhood"])
    for nbhood, grouped in median_lotfrt_per_nbhood:
        index = (df['neighborhood'] == nbhood) & (df['lotfrontage'].isnull())
        df.loc[index,'lotfrontage'] = grouped.median()
    # for the few remaining rows from the neighborhoods without a median (i.e. if n < 3),
    # fill with overall lotfrontage median
    df['lotfrontage'] = df['lotfrontage'].fillna(df['lotfrontage'].median())

    # MASONRY VENEER
    # the NaN values in masvnrarea and masvnrtype are likely houses without masonry veneers
    df['masvnrarea'] = df['masvnrarea'].fillna(0.0)
    df['masvnrtype'] = df['masvnrtype'].fillna('None')
    
    # BASEMENT PROPERTIES
    # NaN values will be replaced with 'None' (no basement present)
    for col in ['bsmtqual','bsmtexposure','bsmtfintype1']:
        df[col] = df[col].fillna('None')
    # for the NaNs in the numerical columns, they will be replaced with 0 (since no basement)
    for col in ['bsmtfullbath','bsmtfinsf1','bsmtunfsf']:
        df[col] = df[col].fillna(0.0)

    # FIREPLACE
    # null value likely represent no fireplace
    df['fireplacequ'] = df['fireplacequ'].fillna('None')
    
    # GARAGE
    # null values likely represent no garage
    for col in ['garagetype','garagefinish','garagecond']:
        df[col] = df[col].fillna('NA')
    for col in ['garagearea','garagecars','garagearea']:
        df[col] = df[col].fillna(0.0)

    # ------------------------------ CONVERT ORDINAL CATEGORIES TO NUMERICAL ------------------------------
    
    # function to map new values
    def map_new_vals(colname,dictionary):
        df[colname] = df[colname].map(dictionary)
    
    # defining dictionaries for renaming
    lotshape_di = {'Reg': 0,
                   'IR1': 1,
                   'IR2': 2,
                   'IR3': 3}
    bsmtex_di = {'None': 0,
                'No': 1,
                'Mn': 2,
                'Av': 3,
                'Gd': 4}
    qual_di = {'Ex': 5,
               'Gd': 4,
               'TA': 3,
               'Fa': 2,
               'Po': 1,
               'None': 0}
    
    # map new values
    map_new_vals('lotshape', dictionary = lotshape_di)
    map_new_vals('bsmtexposure', dictionary = bsmtex_di)
    map_new_vals('exterqual', dictionary = qual_di)
    map_new_vals('bsmtqual', dictionary = qual_di)
    map_new_vals('heatingqc', dictionary = qual_di)
    map_new_vals('kitchenqual', dictionary = qual_di)
    map_new_vals('fireplacequ', dictionary = qual_di)

    # --------------------------------- CONVERT NUMERICAL TO CATEGORICAL --------------------------------
    
    df['mssubclass'] = df['mssubclass'].astype(str)

    # ------------------------------------------ DROP COLUMNS -------------------------------------------
        
    # SKEWED COLUMNS
    # columns with >80% single category or >80% zeros, plus pid columns
    col_to_drop = ['alley','miscval','lowqualfinsf','street','utilities','condition2','roofmatl',\
                   'heating','centralair','electrical','paveddrive','fence','saletype','bsmthalfbath',\
                   'bsmtfintype2','bsmtfinsf2','bsmtcond','extercond','garagequal']
    dropcol(df, col_to_drop)

    # COLLINEAR COLUMNS
    col_collinear = ['garagecars','totrmsabvgrd', 'totalbsmtsf','garageyrblt']
    dropcol(df, col_collinear) 

    # --------------------------------------- REMOVE OUTLIERS -----------------------------------------
    
#     df.drop(df[df['grlivarea'] > 4_500].index, inplace = True)
#     df.drop(df[df['lotfrontage'] > 300].index, inplace = True)
#     df.drop(df[df['lotarea'] > 100_000].index, inplace = True)

    # ----------------------------------------- CLEANING UP -------------------------------------------
    
    # convert all NA to None
    df.replace('NA','None',inplace=True)

    # reset index
    df = df.reset_index()

    return df


In [4]:
# import train
train = pd.read_csv("data/train.csv")

# run preprocessing function
train_preprocessed = preprocess_train(train)

# check df
print(train_preprocessed.columns)
train_preprocessed.info()

# no null values
# on the training data, the preprocessing function retains 2042 out of 2051 samples (99.5%)

Index(['index', 'id', 'mssubclass', 'mszoning', 'lotfrontage', 'lotarea',
       'lotshape', 'landcontour', 'lotconfig', 'landslope', 'neighborhood',
       'condition1', 'bldgtype', 'housestyle', 'overallqual', 'overallcond',
       'yearbuilt', 'yearremodadd', 'roofstyle', 'masvnrtype', 'masvnrarea',
       'exterqual', 'foundation', 'bsmtqual', 'bsmtexposure', 'bsmtfintype1',
       'bsmtfinsf1', 'bsmtunfsf', 'heatingqc', 'grlivarea', 'bsmtfullbath',
       'fullbath', 'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'kitchenqual',
       'functional', 'fireplaces', 'fireplacequ', 'garagetype', 'garagefinish',
       'garagearea', 'garagecond', 'wooddecksf', 'mosold', 'yrsold',
       'salecondition', 'saleprice', 'porchpres', 'poolpres', 'garagepres',
       'fireplacepres', 'masvnrpres', 'bsmtpres', 'shedpres', 'exterior',
       'age_sold', '2ndfloorpres', 'bighouse'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453 entries, 0 to 1452
Data columns (total 5

In [5]:
from scipy.stats import skew

# one zero columns
col_onezero = [col for col in train_preprocessed.columns if len(train_preprocessed[col].unique()) == 2]

# measure skewness of numerical columns (without one zero columns)
col_numerical = [col for col in train_preprocessed.columns if train_preprocessed[col].dtype != np.object]
col_numerical = [col for col in col_numerical if col not in col_onezero]

skewed = train_preprocessed[col_numerical].apply(lambda x: skew(x.astype(float)))

# obtain features with skewness > 0.2
skewed = skewed[abs(skewed) > 0.2]
skewed_df = pd.DataFrame({'skewness_score_original':skewed})

## Transformations - skewness 

The skewness scores below represent the Fisher-Pearson coefficient, which measures of how skewed the data is. Values > 0 means the data is right skewed, and values < 0 means the data is left skewed. Normally distributed data has a skewness of 0.

The graph below compares the results of the log, BoxCox, and square transformation with the skewness of the original data. As expected, the log and Box-Cox transformations made the right skewed features less skewed, but made the left skewed features worse. The opposite was true for square transformation.

For positively skewed features, log transformation will be used as it seemed to have performed slightly better than Box-Cox. For negatively skewed features, the square transformation will be used. The following features will not be transformed as transformation made their skewness score worse: 
- `bsmtunfsf`
- `age_sold`
- `lotfrontage`
- `bedroomsabvgr`
- `mosold`

In [6]:
from scipy.stats import skew

# one zero columns
col_onezero = [col for col in train_preprocessed.columns if len(train_preprocessed[col].unique()) == 2]

# measure skewness of numerical columns (without one zero columns)
col_numerical = [col for col in train_preprocessed.columns if train_preprocessed[col].dtype != np.object]
col_numerical = [col for col in col_numerical if col not in col_onezero]

skewed = train_preprocessed[col_numerical].apply(lambda x: skew(x.astype(float)))

# obtain features with skewness > 0.2
skewed = skewed[abs(skewed) > 0.2]
skewed_df = pd.DataFrame({'skewness_score_original':skewed})


In [7]:
test = train_preprocessed.copy()
skewed_col = skewed.index

# log1p transform (log(1+x))
for col in skewed_col:
    test[col] = np.log1p(test[col])

# result of log transformation
results = test[skewed_col].apply(lambda x: skew(x.astype(float)))
skewed_df['log_transformed'] = results

In [8]:
test = train_preprocessed.copy()

for col in skewed_col:
    test[col] = np.square(test[col])

# results of square transformation
results = test[skewed_col].apply(lambda x: skew(x.astype(float)))
skewed_df['squared'] = results

In [9]:
## Final preprocessing 

In [10]:
# SQUARE

# import data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# preprocessing
train_preprocessed = preprocess_train(train)
test_preprocessed = preprocess_test(test)

# transformation
log_transform = ['kitchenabvgr','lotarea','masvnrarea','wooddecksf','mssubclass','lotshape',\
             'bsmtexposure','grlivarea','bsmtfinsf1','exterqual','halfbath','fireplaces',\
             'overallcond','bsmtfullbath','kitchenqual'] 
sq_transform = ['heatingqc','yearbuilt','bsmtqual']

for col in sq_transform:
    train_preprocessed[col] = np.square(train_preprocessed[col])
    test_preprocessed[col] = np.square(test_preprocessed[col])
    
# train_preprocessed['saleprice'] = np.log1p(train_preprocessed['saleprice'])

## Export 

In [11]:
train_preprocessed.to_csv('data/train_cleaned.csv')
test_preprocessed.to_csv('data/test_cleaned.csv')